<a href="https://colab.research.google.com/github/Kamalapriya-17/Video-Game-Analysis/blob/main/Games.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
ls

sample_data/


In [ ]:
import pandas as pd
import re
import ast
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelEncoder


df=pd.read_csv('games.csv')

df.columns = df.columns.str.lower().str.replace(' ', '_')

def convert_k_to_number(x):
    if isinstance(x, str) and 'K' in x:
        return float(x.replace('K', '')) * 1000
    return x

for col in ['times_listed', 'number_of_reviews', 'plays', 'playing', 'backlogs', 'wishlist']:
    if col in df.columns:
        df[col] = df[col].apply(convert_k_to_number)

df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

df['days_since_release'] = (df['release_date'] - df['release_date'].min()).dt.days

df['release_date']= df['days_since_release']

#Duplicates
df.duplicated().sum()
df.drop_duplicates(inplace=True)
#df.to_csv('games.csv',index=False)



#split to numerical and categorical
df_cat=df.select_dtypes(include='object')
df_num=df.select_dtypes(exclude='object')
print(df_cat.columns)
print(df_num.columns)
#find null values

df_num.isnull().sum()
df_cat.isnull().sum()


#find outliers

def outlier_capping(x):
  q1=x.quantile(0.25)
  q3=x.quantile(0.75)
  iqr=q3-q1
  lower=q1-1.5*iqr
  upper=q3+1.5*iqr
  x=x.clip(lower=lower,upper=upper)
  return x

for i in df_num.columns:
  df_num[i]=outlier_capping(df_num[i])

for i in ["title","summary","reviews"] :

    encoder=LabelEncoder()
    df_cat[i]=encoder.fit_transform(df_cat[i])


def parse_genres(genre_string):
    try:
        return ast.literal_eval(genre_string)
    except (ValueError, SyntaxError):
        return []

print(df_cat['genres'])
df_cat['genres'] = df_cat['genres'].apply(parse_genres)

mlb = MultiLabelBinarizer()
genres_encoded = pd.DataFrame(mlb.fit_transform(df_cat['genres']), columns=mlb.classes_, index=df.index)

df_cat = df_cat.join(genres_encoded)
df_cat = df_cat.drop('genres', axis=1)

Index(['title', 'team', 'times_listed', 'number_of_reviews', 'genres',
       'summary', 'reviews', 'plays', 'playing', 'backlogs', 'wishlist'],
      dtype='object')
Index(['unnamed:_0', 'release_date', 'rating', 'days_since_release'], dtype='object')
0                                    ['Adventure', 'RPG']
1                ['Adventure', 'Brawler', 'Indie', 'RPG']
2                                    ['Adventure', 'RPG']
3       ['Adventure', 'Indie', 'RPG', 'Turn Based Stra...
4                      ['Adventure', 'Indie', 'Platform']
                              ...                        
1507                     ['Adventure', 'Point-and-Click']
1508                                 ['Arcade', 'Racing']
1509                                   ['Brawler', 'RPG']
1510    ['Adventure', 'RPG', 'Strategy', 'Tactical', '...
1511                              ['Adventure', 'Puzzle']
Name: genres, Length: 1512, dtype: object


In [ ]:
merged_df = df_num.join(df_cat)

merged_df.to_csv('cleaned_games.csv', index=False)

print("Merged DataFrame created and saved to 'cleaned_games.csv'")

Merged DataFrame created and saved to 'cleaned_games.csv'
